In [33]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [45]:
df = pd.read_excel("../DataScraper/NBAScores.xlsx")
df.head()

,Unnamed: 0,Team,Match Up,Game Date,W/L,MIN,PTS,FGM,FGA,FG%,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,0,MIA,MIA vs. ORL,04/09/2023,W,240,123,45,83,54.2,...,75.0,7,37,44,30,10,3,18,20,13
1,1,CLE,CLE vs. CHA,04/09/2023,L,240,95,35,89,39.3,...,93.8,11,33,44,27,9,5,16,24,-11
2,2,CHA,CHA @ CLE,04/09/2023,W,240,106,37,81,45.7,...,86.7,10,39,49,23,8,5,19,14,11
3,3,WAS,WAS vs. HOU,04/09/2023,L,240,109,41,97,42.3,...,61.9,14,37,51,28,11,5,13,21,-5
4,4,CHI,CHI vs. DET,04/09/2023,W,240,103,40,95,42.1,...,77.3,11,37,48,21,15,7,4,16,22


We will drop the first three columns and min and final points column as they arn't important. We will also transform the game date as year, since the specific date isn't important.

In [46]:
df.drop(columns=df.columns[0:3], axis=1,  inplace=True)
df.drop(columns=df.columns[2:4], axis=1,  inplace=True)
df.drop(columns=df.columns[-1], axis=1,  inplace=True)
df.head()

,Game Date,W/L,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF
0,04/09/2023,W,45,83,54.2,18,44,40.9,15,20,75.0,7,37,44,30,10,3,18,20
1,04/09/2023,L,35,89,39.3,10,37,27.0,15,16,93.8,11,33,44,27,9,5,16,24
2,04/09/2023,W,37,81,45.7,6,32,18.8,26,30,86.7,10,39,49,23,8,5,19,14
3,04/09/2023,L,41,97,42.3,14,45,31.1,13,21,61.9,14,37,51,28,11,5,13,21
4,04/09/2023,W,40,95,42.1,6,30,20.0,17,22,77.3,11,37,48,21,15,7,4,16


In [47]:
def strip_last_four(s):
    return s[-4:]

df['Game Date'] = df['Game Date'].apply(strip_last_four)
df = df.rename(columns={'Game Date': 'Year'})
df.head()

,Year,W/L,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF
0,2023,W,45,83,54.2,18,44,40.9,15,20,75.0,7,37,44,30,10,3,18,20
1,2023,L,35,89,39.3,10,37,27.0,15,16,93.8,11,33,44,27,9,5,16,24
2,2023,W,37,81,45.7,6,32,18.8,26,30,86.7,10,39,49,23,8,5,19,14
3,2023,L,41,97,42.3,14,45,31.1,13,21,61.9,14,37,51,28,11,5,13,21
4,2023,W,40,95,42.1,6,30,20.0,17,22,77.3,11,37,48,21,15,7,4,16


Seperate Wins and losses into seperate dataframes

In [48]:
df = df.rename(columns={'W/L': 'WinLoss'})
wins = df[df.WinLoss=='W']
losses = df[df.WinLoss=='L']

,Year,WinLoss,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF
1,2023,L,35,89,39.3,10,37,27.0,15,16,93.8,11,33,44,27,9,5,16,24
3,2023,L,41,97,42.3,14,45,31.1,13,21,61.9,14,37,51,28,11,5,13,21
5,2023,L,31,85,36.5,9,33,27.3,10,13,76.9,15,44,59,20,1,3,25,16
6,2023,L,46,108,42.6,17,53,32.1,8,16,50.0,8,38,46,28,4,3,4,21
9,2023,L,35,83,42.2,12,32,37.5,23,28,82.1,10,32,42,22,6,6,19,22


FGM Analysis